In [1]:
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate,
)


In [2]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1e-2)


In [3]:
system_message = """
Your job is to determine the payout for an injury, if applicable. \
Use the following rules to determine the payout:

1. For an injury involving the lower body, the payout is 500 SEK.
2. If the injury is on the left arm, the payout is 300 SEK.
3. A right arm injury does not qualify for a payout, i.e., the payout is 0 SEK.
4. If the person has received a blow to the head, the payout is 1000 SEK.

Please note that an injury might not be covered by the above rules. \
In such cases, inform the individual that you need to consult with your colleagues before proceeding. \
If there are multiple injuries, provide the total payout in your response. \
It's essential that you always refer to the relevant rule number when communicating the payout for an injury.
""".strip()
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

print(system_message_prompt.format().content)


Your job is to determine the payout for an injury, if applicable. Use the following rules to determine the payout:

1. For an injury involving the lower body, the payout is 500 SEK.
2. If the injury is on the left arm, the payout is 300 SEK.
3. A right arm injury does not qualify for a payout, i.e., the payout is 0 SEK.
4. If the person has received a blow to the head, the payout is 1000 SEK.

Please note that an injury might not be covered by the above rules. In such cases, inform the individual that you need to consult with your colleagues before proceeding. If there are multiple injuries, provide the total payout in your response. It's essential that you always refer to the relevant rule number when communicating the payout for an injury.


In [4]:
example_human_message = """
During my bike ride to Grandma's, I fell and broke my leg.
""".strip()
example_human_message_prompt = SystemMessagePromptTemplate.from_template(
    example_human_message,
    additional_kwargs={"name": "example_user"},
)

example_ai_message = """
I'm sorry to hear that. Based on rule #1, you're entitled to a payout of 500 SEK for the injury to your leg.
""".strip()
example_ai_message_prompt = SystemMessagePromptTemplate.from_template(
    example_ai_message,
    additional_kwargs={"name": "example_assistant"},
)

print(example_human_message_prompt.format().content)
print(example_ai_message_prompt.format().content)


During my bike ride to Grandma's, I fell and broke my leg.
I'm sorry to hear that. Based on rule #1, you're entitled to a payout of 500 SEK for the injury to your leg.


In [5]:
human_message = "{injury}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message)

chat_messages = [
    system_message_prompt,
    example_human_message_prompt,
    example_ai_message_prompt,
    human_message_prompt,
]

chat_prompt = ChatPromptTemplate.from_messages(chat_messages)
chain = LLMChain(llm=chat, prompt=chat_prompt)


In [6]:
def decide_payout(injury: str) -> str:
    with get_openai_callback() as cb:
        response = chain.run(injury)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")
        return response


In [7]:
decide_payout("I was hit on the head by a falling brick during construction work.")

Total Tokens: 290
Total Cost (USD): $0.0005800000000000001


"I'm sorry to hear that. Based on rule #4, you're entitled to a payout of 1000 SEK for the injury to your head."

In [8]:
decide_payout("A dinosaur bit off my right arm during the filming of Jurassic Park X.")


Total Tokens: 300
Total Cost (USD): $0.0006000000000000001


"I'm sorry to hear that. Based on rule #3, injuries to the right arm do not qualify for a payout. Therefore, I'm afraid you're not entitled to any payout for this injury."

In [9]:
decide_payout("A dinosaur bit off my left arm during the filming of Jurassic Park X.")


Total Tokens: 330
Total Cost (USD): $0.00066


"I'm sorry to hear that. Based on rule #2, you're entitled to a payout of 300 SEK for the injury to your left arm. However, please note that this is not a complete payout for such a severe injury. I need to consult with my colleagues before proceeding. Can you please provide me with more details about the incident?"

In [10]:
decide_payout("I got hit by a car.")


Total Tokens: 268
Total Cost (USD): $0.000536


"I'm sorry to hear that. Can you please tell me where you were injured?"

In [11]:
decide_payout("Love shattered my heart into pieces.")


Total Tokens: 289
Total Cost (USD): $0.0005780000000000001


"I'm sorry to hear that, but I'm afraid that this situation is not covered by the rules for injury payouts. If you have any other questions or concerns, please let me know."

In [12]:
decide_payout("I have injured both my left arm and my right knee.")


Total Tokens: 316
Total Cost (USD): $0.000632


"I'm sorry to hear that. Based on rule #1 and #2, you're entitled to a payout of 500 SEK for the injury to your knee and 300 SEK for the injury to your left arm. Therefore, the total payout for your injuries is 800 SEK."

In [13]:
decide_payout("Jag har brutit benet. Aj aj vad ont det gjorde och det här skriver jag ju helt på svenska 😃")


Total Tokens: 329
Total Cost (USD): $0.0006580000000000001


'Jag beklagar att du har brutit benet. Enligt regel #1 är du berättigad till en utbetalning på 500 SEK för skadan på ditt ben. Hoppas att du känner dig bättre snart!'